In [16]:
from notebook_utils import resolve_paths_from_parent_directory
resolve_paths_from_parent_directory()
# auto reload notebook deps
%reload_ext autoreload
%autoreload 2

In [17]:
import json
import pandas as pd

DATASETS = {
    "CentralParkNYC": {
        "input_location": "../data/tweet_objects/CentralParkNYC/2021-01-27-2021-02-06.json",
        "output_location": "../data/labeled_datasets/CentralParkNYC-2021-01-27-2021-02-06.json",
        "spike_start": "Feb 1, 2021, 17:00",
        "spike_end": "Feb 3, 2021, 04:00",
    },
    "united": {
        "input_location": "../data/tweet_objects/united/2020-12-05-2020-12-15.json",
        "output_location": "../data/labeled_datasets/united-2020-12-05-2020-12-15.json",
        "spike_start": "Dec 12, 2020, 05:00",
        "spike_end": "Dec 13, 2020, 06:00",
    },
    "nationalGridUS": {
        "input_location": "../data/tweet_objects/nationalGridUS/2020-10-01-2020-10-12.json",
        "output_location": "../data/labeled_datasets/nationalGridUS-2020-10-01-2020-10-12.json",
        "spike_start": "Oct 7, 2020, 17:00",
        "spike_end": "Oct 9, 2020, 04:00",
    }
}

selected_dataset = DATASETS["CentralParkNYC"]
labeled_spike_start = pd.to_datetime(selected_dataset["spike_start"]).tz_localize("UTC")
labeled_spike_end = pd.to_datetime(selected_dataset["spike_end"]).tz_localize("UTC")

In [18]:
from utils.dataset import create_tweet_df

with open(selected_dataset["input_location"], "r") as f:
    raw_dataset = json.load(f)

df_tweets = create_tweet_df(raw_dataset["tweets"])

# Label tweets
df_tweets["is_anomaly"] = df_tweets.created_at.apply(
    lambda x: 0 if (x < labeled_spike_start or x > labeled_spike_end) else 1
)

df_tweets.head()

,id,text,created_at,hashtags,mentions,in_reply_to_user_id,user_id,retweet_count,quote_count,reply_count,like_count,retweeted,replied_to,quoted,is_anomaly
3616,1354218419441315840,With today’s winter weather—and more in the fo...,2021-01-27 00:03:26+00:00,[CentralParkConservancy],[],None,96795310,26,3,3,109,NaN,NaN,NaN,0
3615,1354218565977706498,RT @CentralParkNYC: With today’s winter weathe...,2021-01-27 00:04:00+00:00,[],[CentralParkNYC],None,1288324520487669760,26,0,0,0,1354218419441315840,NaN,NaN,0
3614,1354219142048440323,@CentralParkNYC https://t.co/0DTEzyA1xx,2021-01-27 00:06:18+00:00,[],[CentralParkNYC],96795310,119252795,0,0,0,4,NaN,1354218419441315840,NaN,0
3613,1354219554138968065,RT @CentralParkNYC: With today’s winter weathe...,2021-01-27 00:07:56+00:00,[],[CentralParkNYC],None,151539920,26,0,0,0,1354218419441315840,NaN,NaN,0
3612,1354226585956716548,RT @CentralParkNYC: With today’s winter weathe...,2021-01-27 00:35:53+00:00,[],[CentralParkNYC],None,1342261723793907713,26,0,0,0,1354218419441315840,NaN,NaN,0


In [19]:
# Export 
df_tweets.to_json(
    selected_dataset["output_location"],
    orient="records"
)

In [20]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3617 entries, 3616 to 0
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   id                   3617 non-null   object             
 1   text                 3617 non-null   object             
 2   created_at           3617 non-null   datetime64[ns, UTC]
 3   hashtags             3617 non-null   object             
 4   mentions             3617 non-null   object             
 5   in_reply_to_user_id  970 non-null    object             
 6   user_id              3617 non-null   object             
 7   retweet_count        3617 non-null   int64              
 8   quote_count          3617 non-null   int64              
 9   reply_count          3617 non-null   int64              
 10  like_count           3617 non-null   int64              
 11  retweeted            2302 non-null   object             
 12  replied_to          